# Tutorial 1 - A simple pipeline to plot a signal

First we define a simple pipeline of three agents. Two agents will generate a signal
(in our case based on the *SineGeneratorAgent*), one of which is a default signal and
one with customized parameters. The third one plots the signals on the dashboard,
thus needs to be of type *MonitorAgent* or *MetrologicalMonitorAgent*. Besides, the
outputs of the signal agents need to be bound to the *MonitorAgent*.


Each agent has an internal `current_state` which can be used as a switch to change the 
behaviour of the agent. The available states are listed
[here](https://github.com/Met4FoF/agentMET4FOF/blob/a7e0007fcd460458d629546c99d68ef1f2079c11/agentMET4FOF/agents/base_agents.py#L160).

As soon as all agents are initialized and the connections are set up, the agent 
network is started by accordingly changing all agents' state simultaneously.

In [2]:
# %load tutorial_1_generator_agent.py
from time import sleep

import numpy as np

from agentMET4FOF.agents import AgentNetwork, MonitorAgent, SineGeneratorAgent


def demonstrate_generator_agent_use() -> AgentNetwork:
    # Start agent network server.
    agent_network = AgentNetwork()

    # Initialize agents by adding them to the agent network.
    default_sine_agent = agent_network.add_agent(
        name="Default Sine generator", agentType=SineGeneratorAgent
    )

    # Custom parameters of the specified agentType can either be handed over as **kwargs
    # in the network instance's add_agent() method, or...
    custom_sine_agent = agent_network.add_agent(
        name="Custom Sine generator",
        agentType=SineGeneratorAgent,
        sfreq=75,
        sine_freq=np.pi,
    )
    # ... in a separate call of the agent instance's init_parameters() method.
    custom_sine_agent.init_parameters(
        amplitude=0.75,
    )

    monitor_agent = agent_network.add_agent(
        name="Showcase a default and a customized sine signal", agentType=MonitorAgent
    )

    # Interconnect agents by either way:
    # 1) by agent network.bind_agents(source, target).
    agent_network.bind_agents(default_sine_agent, monitor_agent)

    # 2) by the agent.bind_output().
    custom_sine_agent.bind_output(monitor_agent)

    # Set all agents' states to "Running".
    agent_network.set_running_state()

    # Allow for shutting down the network after execution
    return agent_network


if __name__ == "__main__":
    signal_demo_network = demonstrate_generator_agent_use()
    sleep(60)
    signal_demo_network.shutdown()


Error on connecting to existing name server at http://0.0.0.0:3333: Could not locate the name server!
Starting NameServer...
Broadcast server running on 0.0.0.0:9091
NS running on 0.0.0.0:3333 (0.0.0.0)
URI = PYRO:Pyro.NameServer@0.0.0.0:3333
INFO [2022-02-03 21:05:44.331772] (Default_Sine_generator): INITIALIZED
INFO [2022-02-03 21:05:44.360827] (Custom_Sine_generator): INITIALIZED
INFO [2022-02-03 21:05:44.386817] (Showcase_a_default_and_a_customized_sine_signal): INITIALIZED
[2022-02-03 21:05:44.399664] (Default_Sine_generator): Connected output module: Showcase_a_default_and_a_customized_sine_signal
[2022-02-03 21:05:44.405280] (Custom_Sine_generator): Connected output module: Showcase_a_default_and_a_customized_sine_signal

|------------------------------------------------------------|
|                                                            |
| Your agent network is starting up. Open your browser and   |
| visit the agentMET4FOF dashboard on http://127.0.0.1:8050/ |
|        